In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

In [2]:
import time
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from cxlayers import ImageToElectricField, CxMO, AngularSpectrum, CxD2NNIntensity, D2NNMNISTDetector, ImageResizing, ImageBinarization, D2NNMNISTFilter, CxD2NNFaradayRotation, Polarizer

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [10]:
class MOD2NN(tf.keras.models.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    # 複数モデルを入れ子にすることもOK
    self.model = self.create_model()
    # トラッカーを用意する（訓練、テスト共通で良い）
    self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.loss_tracker = tf.keras.metrics.SparseCategoricalCrossentropy()
    self.accuracy_tracker = tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")

  def create_model(self):
    shape = (28, 28)
    inputs = tf.keras.layers.Input((28, 28))
    x = ImageToElectricField(shape)(inputs)
    x = CxMO(shape)(x)
    x = AngularSpectrum(shape, 633e-9, 0.1e-3, d=0.5e-6, n=1.0, method='band_limited')(x)
    x = CxD2NNIntensity(shape)(x)
    x = D2NNMNISTDetector(10)(x)
    return tf.keras.models.Model(inputs, x)

  # なくてもエラーは出ないが、訓練・テスト間、エポックの切り替わりで
  # トラッカーがリセットされないため、必ずmetricsのプロパティをオーバーライドすること
  # self.reset_metrics()はこのプロパティを参照している
  @property
  def metrics(self):
    return [self.loss_tracker, self.accuracy]

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      pred = self.model(x)
      loss_value = self.loss_object(y, pred)
    # 全体（self）に対する偏微分か、特定モデル（self.model）に対する微分かは場合により変わる
    # このケースではどちらでも同じだが、GANでは使い分ける必要がある
    grads = tape.gradient(loss_value, self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    # エポックの切り替わりのトラッカーのリセットは、self.reset_metrics()で自動的に行われる
    self.loss_tracker.update_state(y, pred)
    self.accuracy_tracker.update_state(y, pred)

    return {
      "loss": self.loss_tracker.result(),
      "accuracy": self.accuracy_tracker.result()
    }

  def test_step(self, data):
    x, y = data

    pred = self.model(x)
    loss = self.loss_object(y, pred)

    # 訓練・テストの切り替わりのトラッカーのリセットは、self.reset_metrics()で自動的に行われる
    self.loss_tracker.update_state(loss)
    return {
      "loss": self.loss_tracker.result()
    }

In [11]:
model = MOD2NN()
model.compile(optimizer=tf.keras.optimizers.Adam())
model.fit(x_train, y_train, batch_size=64, epochs=10)

Epoch 1/10


2022-04-08 13:53:20.956951: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 53s 56ms/step - loss: 2.5735 - accuracy: 0.0735
Epoch 2/10
938/938 [==============================] - 54s 57ms/step - loss: 2.3246 - accuracy: 0.1335
Epoch 3/10
938/938 [==============================] - 54s 57ms/step - loss: 2.1313 - accuracy: 0.1895
Epoch 4/10
938/938 [==============================] - 54s 57ms/step - loss: 2.0572 - accuracy: 0.2277
Epoch 5/10
938/938 [==============================] - 55s 59ms/step - loss: 2.0352 - accuracy: 0.2543
Epoch 6/10
938/938 [==============================] - 54s 58ms/step - loss: 2.0323 - accuracy: 0.2742
Epoch 7/10
938/938 [==============================] - 54s 57ms/step - loss: 2.0320 - accuracy: 0.2898
Epoch 8/10
938/938 [==============================] - 54s 57ms/step - loss: 2.0307 - accuracy: 0.3019
Epoch 9/10
938/938 [==============================] - 53s 57ms/step - loss: 2.0294 - accuracy: 0.3117
Epoch 10/10
938/938 [==============================] - 52s 56ms/step - loss: 2.0259 - accurac

In [16]:
pred = model(x_train[0:10,:,:])
pred

NotImplementedError: Exception encountered when calling layer "mod2nn_2" (type MOD2NN).

Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.

Call arguments received:
  • inputs=tf.Tensor(shape=(10, 28, 28), dtype=float32)
  • training=None
  • mask=None